In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 'deep-learning-analytics/GrammarCorrector'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=164, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=164,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

## Testing the model on text
text = ''' I want to dedicate this book to my family. Rachel, Connor, and Ben, you are my life. To my 
wonderful mother, Susan, and my sister, Kristy, - thank you for the love and support. I also 
would like to thank the faculty, staff, and students at both Notre Dame of Maryland 
University and Johns Hopkins University for allowing me to fulfill my passion for teaching 
the teachers of today and tomorrow. -Ryan L. Schaaf 
 
I want to dedicate this book to my parents, who always promoted the value of a strong 
education. From installing my pretend classroom in our basement and pretending to be my 
first students, to encouraging me throughout college and graduate school, I am forever 
grateful. To my husband, Nick, thank you for being my first editor and always allowing me to 
bounce ideas off of you. We are a team. Thank you for always supporting me. Ian and Ryan, 
thank you for this unbelievable opportunity and all that you have taught me throughout this 
experience. -Becky Zayas 
 
This book is intended to celebrate the exceptional dedication and courage educators have 
exhibited, and to acknowledge their demonstrated capacity to adapt and innovate in 
extraordinarily challenging and uncertain conditions. Now is the time for us to recognize the 
exceptional role they play, and to empower them with the training, professional development, 
support, and working conditions needed to effectively deploy their talents. For the education 
system to recover from the COVID pandemic requires sustained investment in the well-
being, training, professional development and working conditions of the world’s 71 million 
educators. Education recovery will only be successful if it is conducted hand-in-hand with 
teachers, giving them both voice and agency to participate in the critical change process. - Ian 
Jukes 
    
 '''

print(correct_grammar(text, num_return_sequences=2))

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

def load_model():
    model_name = "bert-base-uncased"  # Replace with the appropriate pre-trained model name
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
    return model, tokenizer

def classify_grammar(sentence, model, tokenizer):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    predicted_class_index = torch.argmax(probabilities).item()
    label_mapping = {0: "Grammatically Correct", 1: "Grammatically Incorrect"}
    predicted_class_label = label_mapping[predicted_class_index]
    return predicted_class_label

# Example usage
sentence = "She don't likes ice cream.."

# "She don't likes ice cream."
# "He plays the piano and tennis."
# "My dogs eat cookies and eats cake."

model, tokenizer = load_model()
predicted_label = classify_grammar(sentence, model, tokenizer)
print(f"Sentence: {sentence}")
print(f"Predicted Label: {predicted_label}")


In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from keras.utils import pad_sequences
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
import io
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm , trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from langchain.text_splitter import TokenTextSplitter
import nltk
# nltk.download('punkt')  # Download the punkt tokenizer for sentence tokenization
from nltk.tokenize import sent_tokenize

: 

In [ ]:

# Load the saved state dictionary into the model
model_file = '/home/ubuntu/ritesh_manchikanti/cat_poc_model_2/bert-based-uncased-GED.pth'
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.load_state_dict(torch.load(model_file))
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


In [ ]:

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        resource_manager = PDFResourceManager()
        fake_file_handle = io.StringIO()
        converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
        page_interpreter = PDFPageInterpreter(resource_manager, converter)
        
        for page in PDFPage.get_pages(file, check_extractable=True):
            page_interpreter.process_page(page)
            text += fake_file_handle.getvalue()
        
        converter.close()
        fake_file_handle.close()
    
    return text

# Function to divide text into chunks of 512 tokens
def divide_text_into_chunks(text, max_chunk_size=512):
    tokenized_texts = text.split()  # Assuming the text is already tokenized by spaces, adjust as needed
    chunks = []
    current_chunk = ""
    
    for tokenized_text in tokenized_texts:
        if len(current_chunk) + len(tokenized_text) <= max_chunk_size:
            current_chunk += tokenized_text + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = tokenized_text + " "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks
text_splitter_1 = TokenTextSplitter(chunk_size=100,chunk_overlap = 2)

# Example usage
pdf_path = './15031-4983-FullBook.pdf'
text = extract_text_from_pdf(pdf_path)
chunks = text_splitter_1.split_text(text)
# chunks = divide_text_into_chunks(text)

# Predict each chunk

for idx, chunk in enumerate(chunks[:1]):
    print(len(chunk.split()))
    sentences = sent_tokenize(chunk)
    print(sentences)
    # print(idx,chunk)
    # for sent in chunk:
        # print(sent)
    # tokenized_texts = [tokenizer.tokenize(sent) for sent in chunk]
    
# for idx, chunk in enumerate(chunks[:1]):
#     sentences = sent_tokenize(chunk)
#     # We need to add special tokens at the beginning and end of each sentence
#     # for BERT to work properly
#     sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
#     labels = [0]  # Assuming you have label information for each sentence, you can modify this accordingly

#     tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    
#     tokenized_texts = [tokenizer.tokenize(sent) for sent in chunk]
#     MAX_LEN = 512
#     input_ids = pad_sequences(
#         [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], 
#         maxlen=MAX_LEN, dtype="long", truncating="post", padding="post"
#     )
    
#     # Convert to tensors
#     prediction_inputs = torch.tensor(input_ids)
#     prediction_masks = torch.tensor([[float(i > 0) for i in seq] for seq in input_ids])
    
#     # Put model in evaluation mode
#     model.eval()
    
#     with torch.no_grad():
#         # Forward pass, calculate logit predictions
#         logits = model(prediction_inputs.to(device), token_type_ids=None, attention_mask=prediction_masks.to(device))
    
#     # Move logits to CPU and store predictions for each sentence
#     logits = logits.detach().cpu().numpy()
#     flat_predictions = np.argmax(logits, axis=1).flatten()
    
#     # Print the predictions for each sentence in the chunk
#     for i, predicted_class_index in enumerate(flat_predictions):
#         label_mapping = {0: "Grammatically Correct", 1: "Grammatically Incorrect"}
#         predicted_class_label = label_mapping[predicted_class_index]
#         print(f"Chunk {idx+1} - Sentence {i+1}: {predicted_class_label}")
  


In [ ]:
# import torch
# from transformers import BertForSequenceClassification, BertTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm , trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
# Tokenize Inputs

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case = True)
# Replace 'path_to_model_file.pth' with the path to the saved .pth model file.
model_file = '/home/ubuntu/ritesh_manchikanti/cat_poc_model_2/bert-based-uncased-GED.pth'

# Create an instance of the model's class
# model = BertForSequenceClassification()

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                      num_labels=2)
# Load the saved state dictionary into the model
model.load_state_dict(torch.load(model_file))

# Optional: If you want to use the model for inference, move it to the appropriate device (GPU or CPU).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# run on a sample text

model.eval()

# Create sentence) and label lists
sentences = ["They drank the pub."]
# We need to add special tokens at the beginning and end of each sentence
# for BERT to work properly
print("sentences")
print(sentences)
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels =[0]

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print("tokenized_texts")
print(tokenized_texts)
# Padding Sentences
# Set the maximum sequence length. The longest sequence in our training set
# is 47, but we'll leave room on the end anyway.
# In the original paper, the authors used a length of 512.
MAX_LEN = 128
predictions = []
# Pad our input tokens
input_ids = pad_sequences(
    [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], 
    maxlen=MAX_LEN, dtype="long", truncating="post", padding="post"
    )
# Index Numbers and Padding
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# pad sentences
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype ="long", truncating="post",padding ="post")
# Attention masks
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i > 0) for i in seq]
  attention_masks.append(seq_mask)
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
# Put model in evaluation mode
model.eval()
with torch.no_grad():
  # Forward pass, calculate logit predictions
  logits = model(prediction_inputs.to(device), token_type_ids=None, attention_mask=prediction_masks.to(device))
# Move logits and labels to CPU
logits = logits.detach().cpu().numpy()
# label_ids = b_labels.to("cpu").numpy()
# Store predictions and true labels
predictions.append(logits)
# true_labels.append(label_ids)
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# flat_true_labels = [item for sublist in true_labels for item in sublist]
print(flat_predictions[0])